In [2]:
import pandas as pd

In [2]:
housing_data_unc = pd.read_excel('property_data.xlsx')
housing_data_unc.head()

,ID,NAME,HOST_ID,HOST_NAME,NYC_BOROUGH,NEIGHBORHOOD,LATITUDE,LONGITUDE,INSTANT_BOOKABLE,CANCELLATION_POLICY,ROOM_TYPE,CONSTRUCTION_YEAR,PRICE,SERVICE_FEE,MINIMUM_NIGHTS,NUMBER_OF_REVIEWS,LAST_REVIEW,REVIEWS_PER_MONTH,HOUSE_RULES
0,1001254,Clean & quiet apt home by the park,80014485718,Madaline,Brooklyn,Kensington,40.64749,-73.97237,False,strict,Private room,2020.0,966,193,10.0,9.0,10/19/2021,0.21,Clean up and treat the home the way you'd like...
1,1002102,Skylit Midtown Castle,52335172823,Jenna,Manhattan,Midtown,40.75362,-73.98377,False,moderate,Entire home/apt,2007.0,142,28,30.0,45.0,5/21/2022,0.38,Pet friendly but please confirm with me if the...
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,Elise,Manhattan,Harlem,40.80902,-73.94190,True,flexible,Private room,2005.0,620,124,3.0,0.0,NaN,NaN,"I encourage you to use my kitchen, cooking and..."
3,1002755,NO-NAME,85098326012,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,True,moderate,Entire home/apt,2005.0,368,74,30.0,270.0,7/5/2019,4.64,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,False,moderate,Entire home/apt,2009.0,204,41,10.0,9.0,11/19/2018,0.10,"Please no smoking in the house, porch or on th..."


In [3]:
housing_data_unc.isna().sum()

ID                         0
NAME                       0
HOST_ID                    0
HOST_NAME                406
NYC_BOROUGH               24
NEIGHBORHOOD              16
LATITUDE                   7
LONGITUDE                  7
INSTANT_BOOKABLE           0
CANCELLATION_POLICY       76
ROOM_TYPE                  0
CONSTRUCTION_YEAR        213
PRICE                      0
SERVICE_FEE                0
MINIMUM_NIGHTS           409
NUMBER_OF_REVIEWS        183
LAST_REVIEW            15893
REVIEWS_PER_MONTH      15879
HOUSE_RULES            52131
dtype: int64

In [4]:
def four_var_clean(df):
    df.CONSTRUCTION_YEAR.fillna(df.CONSTRUCTION_YEAR.median(), inplace=True)
    df.REVIEWS_PER_MONTH.fillna(round(df.REVIEWS_PER_MONTH.mean(), 2), inplace=True)
    df.NUMBER_OF_REVIEWS .fillna(int(df.NUMBER_OF_REVIEWS.mean()), inplace=True)


    df.LAST_REVIEW = pd.to_datetime(df.LAST_REVIEW)
    df.LAST_REVIEW = (df.LAST_REVIEW - pd.Timestamp('1970-01-01')) / pd.Timedelta('1D')
    median_date = df.LAST_REVIEW.median()
    df.LAST_REVIEW.fillna(median_date, inplace=True)
    df.LAST_REVIEW = pd.to_datetime(df.LAST_REVIEW, unit='D')
    
    return(df)

In [5]:
housing_data = four_var_clean(housing_data_unc)

In [6]:
housing_data.isna().sum()

ID                         0
NAME                       0
HOST_ID                    0
HOST_NAME                406
NYC_BOROUGH               24
NEIGHBORHOOD              16
LATITUDE                   7
LONGITUDE                  7
INSTANT_BOOKABLE           0
CANCELLATION_POLICY       76
ROOM_TYPE                  0
CONSTRUCTION_YEAR          0
PRICE                      0
SERVICE_FEE                0
MINIMUM_NIGHTS           409
NUMBER_OF_REVIEWS          0
LAST_REVIEW                0
REVIEWS_PER_MONTH          0
HOUSE_RULES            52131
dtype: int64

# For property prices in each neighborhood 

In [39]:
bronx_prices = pd.read_excel('Rolling_prices/rollingsales_bronx.xlsx')
statenisland_prices = pd.read_excel('Rolling_prices/rollingsales_statenisland.xlsx')
queens_prices = pd.read_excel('Rolling_prices/rollingsales_queens.xlsx')
manhattan_prices = pd.read_excel('Rolling_prices/rollingsales_manhattan.xlsx')
brooklyn_prices = pd.read_excel('Rolling_prices/rollingsales_brooklyn.xlsx')

In [40]:
def realign_df(df):
    df = df.iloc[3:]
    df.columns = df.iloc[0]
    df = df[1:]
    df.reset_index(drop=True, inplace=True)
    return(df)

def vertical_stack(*dfs):
    # Check for two dfs
    if len(dfs) < 2:
        raise ValueError("At least two dataframes are required.")
    # Vertically stack the dataframes
    stacked_df = pd.concat(dfs, axis=0, ignore_index=True)
    return stacked_df

In [41]:
bronx_prices.columns

Index(['Bronx Rolling Sales.  All Sales From April 2022 - March 2023',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'PTS Sales Data as of 04/13/2023', 'Unnamed: 20'],
      dtype='object')

In [43]:
bronx_prices = realign_df(bronx_prices)
statenisland_prices = realign_df(statenisland_prices)
queens_prices = realign_df(queens_prices)
manhattan_prices = realign_df(manhattan_prices)
brooklyn_prices = realign_df(brooklyn_prices)

neighb_prices = vertical_stack(bronx_prices, statenisland_prices, manhattan_prices, queens_prices, brooklyn_prices)


In [48]:
agg_data = neighb_prices.groupby('NEIGHBORHOOD').agg({'SALE PRICE': 'mean'}).sort_values('SALE PRICE', ascending = False)
agg_data


3,SALE PRICE
NEIGHBORHOOD,
FINANCIAL,2.182236e+07
LONG ISLAND CITY,1.612128e+07
MIDTOWN CBD,9.652291e+06
SOUTHBRIDGE,9.543506e+06
MOTT HAVEN/PORT MORRIS,8.467947e+06
...,...
STAPLETON,3.237489e+05
ARROCHAR-SHORE ACRES,2.827643e+05
PARKCHESTER,2.494217e+05
